In [1]:
import glob
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import rocks
rocks.set_log_level("error")
import time as t


from astropy.time import Time
import requests
import io

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import angular_separation

from fink_utils.sso import spins as finkus

# Get ZTF fit

In [2]:
# Local Configuration
data_fink = '/data/fink/tutorials/sso/spins/'
data_fink = './'

In [3]:
# ZTF filters 1: g, 2: r
filters = {'1': 'g', '2': 'r'}

In [21]:
data_hg1g2hyb = pd.read_parquet(f'{data_fink}data/v2/sso_fink_SHG1G2.parquet') # All
data_hg1g2 = pd.read_parquet(f'{data_fink}data/v2/sso_fink_HG1G2.parquet')
data_hg = pd.read_parquet(f'{data_fink}data/v2/sso_fink_HG.parquet')

In [22]:
data_hg1g2.loc[0,'params']

{'G1_1': 0.8134157061576843,
 'G1_2': 0.5166602730751038,
 'G2_1': 0.17281629145145416,
 'G2_2': 0.2626616060733795,
 'H_1': 15.445298194885254,
 'H_2': 14.625930786132812,
 'chi2red': 1.8131904602050781,
 'errG1_1': 0.26331207156181335,
 'errG1_2': 0.20545722544193268,
 'errG2_1': 0.06861889362335205,
 'errG2_2': 0.046739429235458374,
 'errH_1': 0.16449691355228424,
 'errH_2': 0.1682967245578766,
 'fit': 0.0,
 'maxphase': 0.4936143159866333,
 'minphase': 0.02395608089864254,
 'rms': 0.0790422111749649,
 'rms_1': 0.06988099962472916,
 'rms_2': 0.08530883491039276,
 'status': 2.0}

In [6]:
len(data_hg), len(data_hg1g2), len(data_hg1g2hyb)

(115238, 115238, 115238)

In [7]:
data_hg1g2hyb.columns

Index(['ssnamenr', 'nmeasurements', 'ndays', 'params'], dtype='object')

In [8]:
# print(data_hg1g2hyb.loc[0,'params'])
# data_hg1g2hyb
data_hg1g2hyb.loc[0,'params']

{'G1_1': 0.6859115362167358,
 'G1_2': 0.4637981951236725,
 'G2_1': 0.001725436537526548,
 'G2_2': 0.0735737755894661,
 'H_1': 14.191803932189941,
 'H_2': 13.580851554870605,
 'R': 0.8703248500823975,
 'alpha0': 1.3911526203155518,
 'chi2red': 4.832531929016113,
 'delta0': -1.1554640531539917,
 'errG1_1': 0.6654443144798279,
 'errG1_2': 0.4265346825122833,
 'errG2_1': 0.21528935432434082,
 'errG2_2': 0.14112797379493713,
 'errH_1': 0.4686225652694702,
 'errH_2': 0.34959638118743896,
 'errR': 0.51173335313797,
 'erralpha0': 1.1884618997573853,
 'errdelta0': 0.7696920037269592,
 'fit': 0.0,
 'maxCosLambda': 0.2662152349948883,
 'maxphase': 0.42834874987602234,
 'meanCosLambda': 0.19120392203330994,
 'minCosLambda': 6.568335493284394e-07,
 'minphase': 0.05220196396112442,
 'rms': 0.1726798266172409,
 'rms_1': 0.17637817561626434,
 'rms_2': 0.1699395626783371,
 'status': 2.0}

# Reformat solutions

In [9]:
# HG to data
names_numbers = rocks.identify(data_hg.ssnamenr)
data_hg['number'] = [nn[1] for nn in names_numbers]
data_hg['name'] = [nn[0] for nn in names_numbers]

for filt in filters.keys():
    print(filt)
    data_hg['HG_H_{}'.format(filters[filt])] = data_hg['params'].apply(lambda x: x['H_{}'.format(filt)]).values
    data_hg['HG_dH_{}'.format(filters[filt])] = data_hg['params'].apply(lambda x: x['errH_{}'.format(filt)]).values
    data_hg['HG_G_{}'.format(filters[filt])] = data_hg['params'].apply(lambda x: x['G_{}'.format(filt)]).values
    data_hg['HG_dG_{}'.format(filters[filt])] = data_hg['params'].apply(lambda x: x['errG_{}'.format(filt)]).values
    data_hg['HG_rms_{}'.format(filters[filt])] = data_hg['params'].apply(lambda x: x['rms_{}'.format(filt)]).values
    

data_hg['minphase'] = np.rad2deg(data_hg['params'].apply(lambda x: x['minphase'])).values
data_hg['maxphase'] = np.rad2deg(data_hg['params'].apply(lambda x: x['maxphase'])).values

data_hg['n_days'] = data_hg['ndays']
data_hg['n_obs'] = data_hg['nmeasurements']

data_hg['HG_rms'] = data_hg['params'].apply(lambda x: x['rms']).values
data_hg['HG_chi2red'] = data_hg['params'].apply(lambda x: x['chi2red']).values

# data_hg = data_hg.drop(columns=[ 'nmeasurements', 'ndays', 'params'])

cond = data_hg.name.notna()
data_hg = data_hg[cond].reset_index(drop=True)

ERROR    [rocks] Could not identify '2013 PY06'.

ERROR    [rocks] Could not identify '2005 YV02'.

ERROR    [rocks] Could not identify '2019 XG01'.

ERROR    [rocks] Could not identify '1999 UK05'.

ERROR    [rocks] Could not identify '2020 HP08'.

...

ERROR    [rocks] Could not identify '162P'.

ERROR    [rocks] Could not identify '2009 CC03'.

ERROR    [rocks] Could not identify '2021 LK02'.

ERROR    [rocks] Could not identify '2018 XV05'.

1
2


In [11]:
# HG1G2 to data
for filt in filters.keys():
    print(filt)
    data_hg1g2['HG1G2_H_{}'.format(filters[filt])] = data_hg1g2['params'].apply(lambda x: x['H_{}'.format(filt)]).values
    data_hg1g2['HG1G2_dH_{}'.format(filters[filt])] = data_hg1g2['params'].apply(lambda x: x['errH_{}'.format(filt)]).values
    data_hg1g2['HG1G2_G1_{}'.format(filters[filt])] = data_hg1g2['params'].apply(lambda x: x['G1_{}'.format(filt)]).values
    data_hg1g2['HG1G2_dG1_{}'.format(filters[filt])] = data_hg1g2['params'].apply(lambda x: x['errG1_{}'.format(filt)]).values
    data_hg1g2['HG1G2_G2_{}'.format(filters[filt])] = data_hg1g2['params'].apply(lambda x: x['G2_{}'.format(filt)]).values
    data_hg1g2['HG1G2_dG2_{}'.format(filters[filt])] = data_hg1g2['params'].apply(lambda x: x['errG2_{}'.format(filt)]).values
    data_hg1g2['HG1G2_rms_{}'.format(filters[filt])] = data_hg1g2['params'].apply(lambda x: x['rms_{}'.format(filt)]).values
    
data_hg1g2['HG1G2_rms'] = data_hg1g2['params'].apply(lambda x: x['rms']).values
data_hg1g2['HG1G2_chi2red'] = data_hg1g2['params'].apply(lambda x: x['chi2red']).values
data_hg1g2['HG1G2_status'] = data_hg1g2['params'].apply(lambda x: x['status']).values
data_hg1g2['HG1G2_fit'] = data_hg1g2['params'].apply(lambda x: x['fit']).values

data_hg1g2 = data_hg1g2.drop(columns=['nmeasurements', 'ndays', 'params'])


1
2


In [12]:
# HG1G2hyb to data
for filt in filters.keys():
    print(filt)
    data_hg1g2hyb['HG1G2hyb_H_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['H_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_dH_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['errH_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_G1_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['G1_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_dG1_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['errG1_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_G2_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['G2_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_dG2_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['errG2_{}'.format(filt)]).values
    data_hg1g2hyb['HG1G2hyb_rms_{}'.format(filters[filt])] = data_hg1g2hyb['params'].apply(lambda x: x['rms_{}'.format(filt)]).values
    

data_hg1g2hyb['HG1G2hyb_RA0'] = np.degrees(data_hg1g2hyb['params'].apply(lambda x: x['alpha0']).values)
data_hg1g2hyb['HG1G2hyb_dRA0'] = np.degrees(data_hg1g2hyb['params'].apply(lambda x: x['erralpha0']).values)
data_hg1g2hyb['HG1G2hyb_DEC0'] = np.degrees(data_hg1g2hyb['params'].apply(lambda x: x['delta0']).values)
data_hg1g2hyb['HG1G2hyb_dDEC0'] = np.degrees(data_hg1g2hyb['params'].apply(lambda x: x['errdelta0']).values)
data_hg1g2hyb['HG1G2hyb_R'] = data_hg1g2hyb['params'].apply(lambda x: x['R']).values
data_hg1g2hyb['HG1G2hyb_dR'] = data_hg1g2hyb['params'].apply(lambda x: x['errR']).values

data_hg1g2hyb['HG1G2hyb_rms'] = data_hg1g2hyb['params'].apply(lambda x: x['rms']).values
data_hg1g2hyb['HG1G2hyb_chi2red'] = data_hg1g2hyb['params'].apply(lambda x: x['chi2red']).values
data_hg1g2hyb['HG1G2hyb_status'] = data_hg1g2hyb['params'].apply(lambda x: x['status']).values
data_hg1g2hyb['HG1G2hyb_fit'] = data_hg1g2hyb['params'].apply(lambda x: x['fit']).values
data_hg1g2hyb['HG1G2hyb_minCosLambda'] = data_hg1g2hyb['params'].apply(lambda x: x['minCosLambda']).values
data_hg1g2hyb['HG1G2hyb_meanCosLambda'] = data_hg1g2hyb['params'].apply(lambda x: x['meanCosLambda']).values
data_hg1g2hyb['HG1G2hyb_maxCosLambda'] = data_hg1g2hyb['params'].apply(lambda x: x['maxCosLambda']).values

data_hg1g2hyb = data_hg1g2hyb.drop(columns=['nmeasurements', 'ndays', 'params'])


1
2


In [13]:
# # HG1G2atlas to data
# for filt in filters_w_atlas.keys():
#     print(filt)
#     data_hg1g2atlas['HG1G2hyb_H_{}'.format(filters_w_atlas[filt])] = data_hg1g2atlas['params'].apply(lambda x: x['H_{}'.format(filt)]).values
#     data_hg1g2atlas['HG1G2hyb_dH_{}'.format(filters_w_atlas[filt])] = data_hg1g2atlas['params'].apply(lambda x: x['errH_{}'.format(filt)]).values
#     data_hg1g2atlas['HG1G2hyb_G1_{}'.format(filters_w_atlas[filt])] = data_hg1g2atlas['params'].apply(lambda x: x['G1_{}'.format(filt)]).values
#     data_hg1g2atlas['HG1G2hyb_dG1_{}'.format(filters_w_atlas[filt])] = data_hg1g2atlas['params'].apply(lambda x: x['errG1_{}'.format(filt)]).values
#     data_hg1g2atlas['HG1G2hyb_G2_{}'.format(filters_w_atlas[filt])] = data_hg1g2atlas['params'].apply(lambda x: x['G2_{}'.format(filt)]).values
#     data_hg1g2atlas['HG1G2hyb_dG2_{}'.format(filters_w_atlas[filt])] = data_hg1g2atlas['params'].apply(lambda x: x['errG2_{}'.format(filt)]).values
    

# data_hg1g2atlas['HG1G2hyb_RA0'] = np.degrees(data_hg1g2atlas['params'].apply(lambda x: x['alpha0']).values)
# data_hg1g2atlas['HG1G2hyb_dRA0'] = np.degrees(data_hg1g2atlas['params'].apply(lambda x: x['erralpha0']).values)
# data_hg1g2atlas['HG1G2hyb_DEC0'] = np.degrees(data_hg1g2atlas['params'].apply(lambda x: x['delta0']).values)
# data_hg1g2atlas['HG1G2hyb_dDEC0'] = np.degrees(data_hg1g2atlas['params'].apply(lambda x: x['errdelta0']).values)
# data_hg1g2atlas['HG1G2hyb_R'] = data_hg1g2atlas['params'].apply(lambda x: x['R']).values
# data_hg1g2atlas['HG1G2hyb_dR'] = data_hg1g2atlas['params'].apply(lambda x: x['errR']).values

# data_hg1g2atlas['HG1G2hyb_rms'] = data_hg1g2atlas['params'].apply(lambda x: x['rms']).values
# data_hg1g2atlas['HG1G2hyb_chi2red'] = data_hg1g2atlas['params'].apply(lambda x: x['chi2red']).values
# data_hg1g2atlas['HG1G2hyb_n_oppo'] = data_hg1g2atlas['params'].apply(lambda x: x['noppositions']).values
# data_hg1g2atlas['HG1G2hyb_status'] = data_hg1g2atlas['params'].apply(lambda x: x['status']).values
# data_hg1g2atlas['HG1G2hyb_fit'] = data_hg1g2atlas['params'].apply(lambda x: x['fit']).values
# # data_hg1g2atlas['HG1G2hyb_minphase'] = np.rad2deg(data_hg1g2atlas['params'].apply(lambda x: x['minphase']).values)
# # data_hg1g2atlas['HG1G2hyb_maxphase'] = np.rad2deg(data_hg1g2atlas['params'].apply(lambda x: x['maxphase']).values)
# # data_hg1g2atlas['HG1G2hyb_n_days'] = data_hg1g2atlas['ndays']
# # data_hg1g2atlas['HG1G2hyb_n_obs'] = data_hg1g2atlas['nmeasurements']

# data_hg1g2atlas = data_hg1g2atlas.drop(columns=['nmeasurements', 'ndays', 'params'])


## Write to disk

In [14]:
data_hg

,ssnamenr,nmeasurements,ndays,params,number,name,HG_H_g,HG_dH_g,HG_G_g,HG_dG_g,...,HG_dH_r,HG_G_r,HG_dG_r,HG_rms_r,minphase,maxphase,n_days,n_obs,HG_rms,HG_chi2red
0,95367,82,1037.766238,"{'G_1': 0.06303630024194717, 'G_2': 0.19601918...",95367.0,2002 CS157,15.205434,0.049852,6.303630e-02,0.065679,...,0.034712,0.196019,0.036618,0.079057,2.185349,19.910828,1037.766238,82,0.080174,0.498645
1,250264,64,219.697303,"{'G_1': 0.22324514389038086, 'G_2': 0.31732639...",250264.0,2003 CV12,16.374287,0.171846,2.232451e-01,0.148994,...,0.161842,0.317326,0.148689,0.181994,9.820111,28.903004,219.697303,64,0.170350,5.564550
2,26487,208,1101.873183,"{'G_1': 0.33757442235946655, 'G_2': 0.33143043...",26487.0,2000 AV236,14.556883,0.039258,3.375744e-01,0.040907,...,0.042811,0.331430,0.043293,0.110661,0.892786,29.866382,1101.873183,208,0.112170,5.837453
3,189809,63,1002.917616,"{'G_1': 2.0782619075841241e-16, 'G_2': 0.08747...",189809.0,2002 LO19,15.083442,0.085618,2.078262e-16,0.089987,...,0.064286,0.087478,0.070784,0.161389,1.605850,20.178815,1002.917616,63,0.163775,3.231980
4,34395,116,1087.781169,"{'G_1': 0.5242671966552734, 'G_2': 0.381103187...",34395.0,2000 RS73,15.081876,0.147779,5.242672e-01,0.206629,...,0.111389,0.381103,0.136778,0.247270,2.925089,24.701395,1087.781169,116,0.260464,9.454986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115142,20866,216,1061.878310,"{'G_1': 0.25884151458740234, 'G_2': 0.40618246...",20866.0,2000 VP37,13.469830,0.094156,2.588415e-01,0.089800,...,0.124205,0.406182,0.133361,0.168710,3.767693,23.331139,1061.878310,216,0.164734,9.298336
115143,39790,96,1016.752407,"{'G_1': 0.08864176273345947, 'G_2': 0.03623726...",39790.0,1997 PF,13.892776,0.040307,8.864176e-02,0.039136,...,0.041717,0.036237,0.037341,0.115615,0.713536,20.488185,1016.752407,96,0.112657,1.640129
115144,99263,65,794.646273,"{'G_1': 1.0, 'G_2': 0.8298622369766235, 'H_1':...",99263.0,2001 OZ31,17.481823,0.220211,1.000000e+00,0.380656,...,0.160112,0.829862,0.228391,0.261312,1.161068,33.233003,794.646273,65,0.269517,10.753973
115145,18754,228,1181.814294,"{'G_1': 0.19368864595890045, 'G_2': 0.32351323...",18754.0,1999 GL21,14.145675,0.052304,1.936886e-01,0.048243,...,0.054813,0.323513,0.057257,0.162891,3.290924,28.166682,1181.814294,228,0.160058,9.851058


In [15]:
# HG with HG1G2
data_2 = data_hg.merge( data_hg1g2, on='ssnamenr')

# (HG with HG1G2) with hybrid
data = data_2.merge( data_hg1g2hyb, on='ssnamenr')

In [16]:
len(data), len(data_hg), len(data_hg1g2), len(data_hg1g2hyb)

(115147, 115147, 115238, 115238)

In [17]:
data = data.astype({'number':'Int64'})
data

,ssnamenr,nmeasurements,ndays,params,number,name,HG_H_g,HG_dH_g,HG_G_g,HG_dG_g,...,HG1G2hyb_dDEC0,HG1G2hyb_R,HG1G2hyb_dR,HG1G2hyb_rms,HG1G2hyb_chi2red,HG1G2hyb_status,HG1G2hyb_fit,HG1G2hyb_minCosLambda,HG1G2hyb_meanCosLambda,HG1G2hyb_maxCosLambda
0,95367,82,1037.766238,"{'G_1': 0.06303630024194717, 'G_2': 0.19601918...",95367,2002 CS157,15.205434,0.049852,6.303630e-02,0.065679,...,158.532502,0.922042,0.115025,0.077086,0.497046,2.0,0.0,1.012016e-06,0.139533,0.927224
1,250264,64,219.697303,"{'G_1': 0.22324514389038086, 'G_2': 0.31732639...",250264,2003 CV12,16.374287,0.171846,2.232451e-01,0.148994,...,2.862553,0.100000,0.053895,0.159935,4.771893,2.0,0.0,9.388329e-01,0.988473,0.995369
2,26487,208,1101.873183,"{'G_1': 0.33757442235946655, 'G_2': 0.33143043...",26487,2000 AV236,14.556883,0.039258,3.375744e-01,0.040907,...,36.886276,0.825462,0.183103,0.104603,5.488952,2.0,0.0,1.837519e-06,0.257722,0.568765
3,189809,63,1002.917616,"{'G_1': 2.0782619075841241e-16, 'G_2': 0.08747...",189809,2002 LO19,15.083442,0.085618,2.078262e-16,0.089987,...,12.368218,0.236502,0.116943,0.146654,2.808556,2.0,0.0,6.508819e-01,0.705825,0.802373
4,34395,116,1087.781169,"{'G_1': 0.5242671966552734, 'G_2': 0.381103187...",34395,2000 RS73,15.081876,0.147779,5.242672e-01,0.206629,...,14.341654,0.532330,0.467085,0.254345,9.497408,2.0,0.0,5.012663e-03,0.161519,0.300628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115142,20866,216,1061.878310,"{'G_1': 0.25884151458740234, 'G_2': 0.40618246...",20866,2000 VP37,13.469830,0.094156,2.588415e-01,0.089800,...,18.679753,0.746724,0.027362,0.125369,4.812122,2.0,0.0,2.118916e-03,0.547583,0.936407
115143,39790,96,1016.752407,"{'G_1': 0.08864176273345947, 'G_2': 0.03623726...",39790,1997 PF,13.892776,0.040307,8.864176e-02,0.039136,...,10.597246,0.128320,0.231076,0.071692,1.223396,2.0,0.0,3.269790e-01,0.430643,0.561184
115144,99263,65,794.646273,"{'G_1': 1.0, 'G_2': 0.8298622369766235, 'H_1':...",99263,2001 OZ31,17.481823,0.220211,1.000000e+00,0.380656,...,12.426569,0.436599,0.355626,0.267769,12.144855,2.0,0.0,7.321041e-01,0.885299,0.946877
115145,18754,228,1181.814294,"{'G_1': 0.19368864595890045, 'G_2': 0.32351323...",18754,1999 GL21,14.145675,0.052304,1.936886e-01,0.048243,...,25.098514,0.845688,0.274284,0.159227,9.858122,2.0,0.0,2.312535e-07,0.106539,0.307556


In [18]:
data.to_parquet(f'{data_fink}data/sso_ZTF.parquet')

In [19]:
data.columns

Index(['ssnamenr', 'nmeasurements', 'ndays', 'params', 'number', 'name',
       'HG_H_g', 'HG_dH_g', 'HG_G_g', 'HG_dG_g', 'HG_rms_g', 'HG_H_r',
       'HG_dH_r', 'HG_G_r', 'HG_dG_r', 'HG_rms_r', 'minphase', 'maxphase',
       'n_days', 'n_obs', 'HG_rms', 'HG_chi2red', 'HG1G2_H_g', 'HG1G2_dH_g',
       'HG1G2_G1_g', 'HG1G2_dG1_g', 'HG1G2_G2_g', 'HG1G2_dG2_g', 'HG1G2_rms_g',
       'HG1G2_H_r', 'HG1G2_dH_r', 'HG1G2_G1_r', 'HG1G2_dG1_r', 'HG1G2_G2_r',
       'HG1G2_dG2_r', 'HG1G2_rms_r', 'HG1G2_rms', 'HG1G2_chi2red',
       'HG1G2_status', 'HG1G2_fit', 'HG1G2hyb_H_g', 'HG1G2hyb_dH_g',
       'HG1G2hyb_G1_g', 'HG1G2hyb_dG1_g', 'HG1G2hyb_G2_g', 'HG1G2hyb_dG2_g',
       'HG1G2hyb_rms_g', 'HG1G2hyb_H_r', 'HG1G2hyb_dH_r', 'HG1G2hyb_G1_r',
       'HG1G2hyb_dG1_r', 'HG1G2hyb_G2_r', 'HG1G2hyb_dG2_r', 'HG1G2hyb_rms_r',
       'HG1G2hyb_RA0', 'HG1G2hyb_dRA0', 'HG1G2hyb_DEC0', 'HG1G2hyb_dDEC0',
       'HG1G2hyb_R', 'HG1G2hyb_dR', 'HG1G2hyb_rms', 'HG1G2hyb_chi2red',
       'HG1G2hyb_status', 'HG1G2hy